In [1]:
from faker import Faker
faker = Faker()

In [19]:
currency_symbols = ["$", "XAF"]

def get_expense(limit=None, is_category=False):
    val=100
    items = ["Tote Bags", "Backpacks", "Branded Mugs", "Glasses", "Pens", "Sunglasses"]
    
    if is_category:
        # https://www.freshbooks.com/hub/accounting/categorize-expenses-small-business
        # For more details about expense category reference the link above.
        # https://corporatefinanceinstitute.com/resources/knowledge/accounting/expenses/
        
        val = 1000 # category value
        items = ["Advertising Expenses", "Business Vehicle(s)", "Payroll", "Employee Benefits", "Meals & Entertainment Expenses", "Office Expenses",
                      "Office Supplies", "Professional Services", "Rent, Utilities & Phone", "Travel Expenses"]
    
    if limit and limit!=1:
        try:
            items_name = faker.words(nb=limit, ext_word_list=items, unique=True)

            expense_items_data = []
            for i in range(limit):        
                expense_items_data.append({
                    "name": items_name[i],
                    "currency_sym": faker.word(ext_word_list=currency_symbols),
                    "value": faker.random_int(max=val)
                })

            return expense_items_data
        except ValueError:
            print("Limit can not be a negative number (%d)" %limit)
        except Exception:
            print("Limit can only be an integer value ({})".format(limit))
    
    else:
        return {
            "name": faker.word(ext_word_list=items),
            "currency_sym": faker.word(ext_word_list=currency_symbols),
            "value": faker.random_int(max=val)
        }
    
def get_working_capital():
    return faker.random_int(max=10000) - faker.random_int(max=10000)
        
def get_high_expense(limit=None):
    expense_categories = get_expense(limit=limit, is_category=True)
    
    if limit:
        return sorted(expense_categories, key=lambda x: x["value"], reverse=True)

    return expense_categories

def get_invoice_statistics(invoice_count=10, invoice_value=1000):
    unpaid_invoice_count = unpaid_overdue_invoice_count = paid_invoice_count = invoice_count
    unpaid_invoice_value = unpaid_overdue_invoice_value = paid_invoice_value = invoice_value
    
    return {
            "unpaid": {
                "count": faker.random_int(max=unpaid_invoice_count),
                "value": faker.random_int(max=unpaid_invoice_value),
                "currency_sym": faker.word(ext_word_list=currency_symbols)
            },
            "unpaid_overdue": {
                "count": faker.random_int(max=unpaid_overdue_invoice_count),
                "value": faker.random_int(max=unpaid_overdue_invoice_value),
                "currency_sym": faker.word(ext_word_list=currency_symbols)
            },
            "paid": {
                "count": faker.random_int(max=paid_invoice_count),
                "value": faker.random_int(max=paid_invoice_value),
                "currency_sym": faker.word(ext_word_list=currency_symbols)
            }
        }

def get_ratio():
    ratio_types = ["quick ratio", "current ratio"]
    return {
        "type": faker.word(ext_word_list=ratio_types),
        "value": faker.random.random() * 3
    }

def get_expense_percentage():
    # https://socratic.org/questions/if-250-is-increased-to-300-what-is-the-percentage-increase
    # reference from here.
    
    pervious_expense = faker.random_int(max=1000)
    current_expense = faker.random_int(max=1000)
    
    diff = current_expense - pervious_expense
    percentage_diff = (diff / pervious_expense) * 100
    
    return abs(percentage_diff)

def get_all_customers_debt(limit=10):
    customer_invoice_details = []
    
    for i in range(limit):
        paid = faker.random_int(max=500)
        unpaid = faker.random_int(max=500)
        unpaid_overdue = faker.random_int(max=500)        
        
        customer_invoice_details.append({
            "name": faker.name(),
            "paid_invoice": {"value": paid, "count": faker.random_int(min=1, max=15) if paid else 0},
            "unpaid_invoice": {"value": unpaid, "count": faker.random_int(min=1, max=15) if unpaid else 0},
            "unpaid_overdue_invoice": {"value": unpaid_overdue, "count": faker.random_int(min=1, max=15) if unpaid_overdue else 0},
            "currency_sym": faker.word(ext_word_list=currency_symbols)
        })
        
    return customer_invoice_details
        

def get_customers_invoice_statistics(limit):
    # https://www.percentagecal.com/answer/300-is-what-percent-of-%2010000
    # reference from the link above
    
    customers_invoice_statistics = get_all_customers_debt(limit=limit)
    
    total_paid_sum = sum([customer_invoices["paid_invoice"]["value"] for customer_invoices in customers_invoice_statistics])
    total_unpaid_sum = sum([customer_invoices["unpaid_invoice"]["value"] for customer_invoices in customers_invoice_statistics])
    total_unpaid_overdue_sum = sum([customer_invoices["unpaid_overdue_invoice"]["value"] for customer_invoices in customers_invoice_statistics])
    
    total_overall_debt = total_unpaid_sum + total_unpaid_overdue_sum
    
    
    customer_statistics = []
    for customer_invoices in customers_invoice_statistics:
        customer_statistics.append({
            **customer_invoices,
            "paid_invoice_percentage": (customer_invoices["paid_invoice"]["value"] * 100) / total_paid_sum,
            "unpaid_invoice_percentage": (customer_invoices["unpaid_invoice"]["value"] * 100) / total_unpaid_sum,
            "unpaid_overdue_invoice_percentage": (customer_invoices["unpaid_overdue_invoice"]["value"] * 100) / total_unpaid_overdue_sum,
            "total_overall_unpaid_invoice_percentage": ((customer_invoices["unpaid_invoice"]["value"] + customer_invoices["unpaid_overdue_invoice"]["value"]) * 100) / total_overall_debt,
        })
        
    return {
        "customers": customer_statistics,
        "total_paid_invoice": total_paid_sum,
        "total_unpaid_invoice": total_unpaid_sum,
        "total_unpaid_overdue_invoice": total_unpaid_overdue_sum,
        "currency_sym": faker.word(ext_word_list=currency_symbols)
    }
    
    
def get_highest_invoice_percentage(key):
    # examples of validate keys to use. ["paid_invoice_percentage", "unpaid_invoice_percentage", "unpaid_overdue_invoice_percentage", "total_overall_unpaid_invoice_percentage"]
    
    statistics = get_customers_invoice_statistics(limit=10)
    return max(statistics["customers"], key=lambda x: x[key])

def get_invoice_count(key, is_counter=True):
    # examples of validate keys to use. ["paid_invoice", "unpaid_invoice", "unpaid_overdue_invoice"]
    
    statistics = get_customers_invoice_statistics(limit=10)
    
    if is_counter:
        return sum(customer[key]["count"] for customer in statistics["customers"])
    else:
        return {"total": sum(customer[key]["value"] for customer in statistics["customers"]), "currency_sym": statistics["currency_sym"]}  


def get_top_account_recievable_strategy():
    return [
        "Establishing clear payment terms", 
        "Invoicing effectively", 
        "Offering discounts for early payment", 
        "Making it easy for customers to pay you ex: Flexible payment methods", 
        "Prioritize customers with large balances in the cash collections process", 
        "Define weekly cash collection targets", 
        "Increasing sales and inventory turnover for more details 'How {tense_verb_to_be} my revenue for {period}?'"
    ]


def get_top_account_payable_strategy():
    return [
        "Plan for total spend", 
        "Managing inventory turnover",
        "Drawing up budgets", 
        "Making adjustments for both credit sales and credit purchases, both of which can affect cash flow and your ability to pay vendors early or on-time", 
        "Paying off current liabilities"
    ]


def get_common_business_expense_list():
    return [
        "Advertising", 
        "Bank fees", 
        "Consulting fees", 
        "Professional fees", 
        "Contract labor", 
        "Due and subscriptions", 
        "Mileage", 
        "Office supplies", 
        "Postage", 
        "Printing", 
        "Payroll", 
        "Softwares", 
        "Travels", 
        "Utilities"
    ]


98
{'total': 2087, 'currency_sym': 'XAF'}
